# Membrane Image Segmentation

Import required libraries

In [1]:
import tifffile
import numpy as np

Load the membrane image

In [2]:
image = tifffile.imread('membrane.tif')

FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/git-bob-playground/git-bob-playground/membrane.tif'

Apply threshold of 125

In [ ]:
binary = (image > 125).astype(np.uint8) * 255

Save the binary result

In [ ]:
tifffile.imwrite('membrane_binary.tif', binary)